In [ ]:
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from functools import partial
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,Normalize,RandomBrightnessContrast,
    VerticalFlip,
    Rotate
)
import pandas as pd
#from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
import tensorflow as tf
import numpy as np
import cv2

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Input,Lambda

#from albumentations import *
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
pictureCsv = pd.read_csv("../input/mnist-zeroshot-learning/trainPictureArray.csv")
sigmoidTrain = pd.read_csv("../input/mnist-zeroshot-learning/pictureIndexData.csv")

In [ ]:
train = pd.read_csv("../input/trainvalsplit/trainSplit.csv")
validation = pd.read_csv("../input/trainvalsplit/validationSplit.csv")

In [ ]:
pictureCsvNoLabel = pictureCsv.iloc[:,:-1]
pictureCsvNoLabel = tf.convert_to_tensor(pictureCsvNoLabel.to_numpy())

In [ ]:
firstPictureIndex = train["firstPictureIndex"].values
secondPictureIndex = train["secondPictureIndex"].values

firstPictureIndexValidation = validation["firstPictureIndex"].values
secondPictureIndexValidation = validation["secondPictureIndex"].values

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy()
imageV2 = np.stack([image,image,image])

In [ ]:
pictureCsv

In [ ]:
pixels = image.reshape((28, 28))

        # Plot
plt.title('Label is {label}'.format(label=2))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
transforms = Compose([
            HorizontalFlip(),
            VerticalFlip(),
            #Normalize(),
            #RandomBrightnessContrast(),
           Normalize()
        ])

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
        
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float64)
    return aug_img

def preprocess_siamese(positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [ ]:
imageFinalize = preprocess_image(0)

In [ ]:
image = pictureCsv.iloc[0][:-1].to_numpy().reshape((28,28))
imageV2 = np.stack([image,image,image],axis=2)
#imageV2 = imageV2.reshape((28,28,3))
#imageV2 = np.swapaxes(imageV2,0,2)
resized = cv2.resize(imageV2, (224,224), interpolation = cv2.INTER_AREA)
resized = resized*255

In [ ]:
import cv2

In [ ]:
for i in range(1):  
  plt.subplot(330 + 1 + i)
  plt.imshow(imageV2 , cmap=plt.get_cmap('gray'))
  plt.show()

Add Data Augmentation

In [ ]:
transforms = Compose([
            HorizontalFlip(),
            VerticalFlip(),
           Normalize()
        ])

val_transforms = Compose([
           Normalize()
        ])

In [ ]:
def preprocess_image(index,process_type = "train",target_shape = (224,224)):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image = tf.reshape(pictureCsvNoLabel[index],(28,28))
    image = tf.stack([image, image, image],axis=2)
    #image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.reshape(image,(28,28,3))*255

    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    
    if process_type == "train":
        image = process_Augment_data(image,img_size = target_shape[0])
    elif process_type == "validation":
        image = process_Augment_data_validation(image,img_size = target_shape[0])
    return image

def aug_fn(image, img_size):
    data = {"image":image}
    aug_data = transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def aug_fn_test(image, img_size):
    data = {"image":image}
    aug_data = val_transforms(image=image)
    aug_img = aug_data["image"]
    aug_img = tf.image.resize(aug_img, size=[img_size, img_size])
    return aug_img

def process_Augment_data(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def process_Augment_data_validation(image, img_size):
    aug_img = tf.numpy_function(func=aug_fn_test, inp=[image, img_size], Tout=tf.float32)
    return aug_img

def preprocess_siamese(positive, negative,mode="train"):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(positive,mode),
        preprocess_image(negative,mode),
    )

In [ ]:
def contrastive_loss(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted
	# class label data type (otherwise we run the risk of having two
	# separate data types, causing TensorFlow to error out)
	y = tf.cast(y, preds.dtype)
	# calculate the contrastive loss between the true labels and
	# the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
	# return the computed contrastive loss to the calling function
	return loss

In [ ]:
sigmoidTrain["symmetrylabel"].values

In [ ]:
def create_dataset(listFirstPictureIndex,listSecondPictureIndex,batchsize,labels,useType="train"):
    FirstIndex = listFirstPictureIndex
    SecondIndex = listSecondPictureIndex
    dataset1 = tf.data.Dataset.from_tensor_slices(FirstIndex)
    dataset2 = tf.data.Dataset.from_tensor_slices(SecondIndex)
    labelDatasets = tf.data.Dataset.from_tensor_slices(labels)
    
    dataset = tf.data.Dataset.zip((dataset1, dataset2))
    dataset = dataset.map(lambda x,y:preprocess_siamese(x,y,useType))
    
    dataset = tf.data.Dataset.zip((dataset,labelDatasets))
    dataset = dataset.batch(batchsize, drop_remainder=False).prefetch(8)
    
    return dataset

In [ ]:
train["symmetrylabel"].values

In [ ]:
train_ds = create_dataset(firstPictureIndex,secondPictureIndex,64,train["symmetrylabel"].values,"train")
val_ds = create_dataset(firstPictureIndexValidation,secondPictureIndexValidation,64,validation["symmetrylabel"].values,"validation")

In [ ]:
"""positive_images = firstPictureIndex 
negative_images = secondPictureIndex

image_count = len(positive_images )

positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
label_dataset = tf.data.Dataset.from_tensor_slices(sigmoidTrain["symmetrylabel"].values)
# To generate the list of negative images, let's randomize the list of
# available images and concatenate them together.

#negative_images = triplettrain["Negative"].values
#np.random.RandomState(seed=32).shuffle(negative_images)

#negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
#negative_dataset = negative_dataset.shuffle(buffer_size=4096)

dataset = tf.data.Dataset.zip((positive_dataset, negative_dataset))
#dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.map(preprocess_siamese)

dataset = tf.data.Dataset.zip((dataset,label_dataset))
dataset = dataset.shuffle(buffer_size=1024)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(8)"""

In [ ]:
for data in train_ds.take(3):
  break

In [ ]:
for data in val_ds.take(3):
  break

In [ ]:
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
input1 = tf.keras.Input(shape=(224,224,3))
input2 = tf.keras.Input(shape=(224,224,3))
effinetB0 = EfficientNetB0(include_top=False,weights="imagenet")(input1)
effinetB0_siamese = EfficientNetB0(include_top=False,weights="imagenet")(input2)
pooling1= tf.keras.layers.GlobalAveragePooling2D()(effinetB0)
pooling2 = tf.keras.layers.GlobalAveragePooling2D()(effinetB0_siamese)
distance = Lambda(euclidean_distance)([pooling1, pooling2])

firstEffinet = tf.keras.Model(
    inputs=input1, outputs=pooling1,name = "effinetB0"
)
twinEffinet = tf.keras.Model(
    inputs = input2,outputs= pooling2,name="effinetB0_siamese"
)

inputA = tf.keras.Input(shape=(224,224,3))
inputB = tf.keras.Input(shape=(224,224,3))
featureFirst = firstEffinet(inputA)
featureSecond = twinEffinet(inputB)

distance = Lambda(euclidean_distance)([featureFirst , featureSecond ])
siamese_network = Model(inputs=[inputA,inputB], outputs=distance)

In [ ]:
def contrastive_loss(y, preds, margin=1):
	# explicitly cast the true class label data type to the predicted
	# class label data type (otherwise we run the risk of having two
	# separate data types, causing TensorFlow to error out)
	y = tf.cast(y, preds.dtype)
	# calculate the contrastive loss between the true labels and
	# the predicted labels
	squaredPreds = K.square(preds)
	squaredMargin = K.square(K.maximum(margin - preds, 0))
	loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
	# return the computed contrastive loss to the calling function
	return loss

In [ ]:
siamese_network.summary()

In [ ]:
siamese_network.compile(loss=contrastive_loss, optimizer="adam")

In [ ]:
checkpoint_filepath = "./siamesemodel{epoch:02d}-validationloss{val_loss:.2f}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq = "epoch",
    save_weights_only=True,
    monitor='val_loss',
    save_best_only=False,
    verbose = 1
    )

In [ ]:
siamese_network.fit(train_ds, epochs=10, validation_data=val_ds,callbacks=[model_checkpoint_callback])